In [4]:
!pip install selenium


  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB 1.4 MB/s eta 0:00:07
   - -------------------------------------- 0.3/9.5 MB 5.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.3/9.5 MB 11.7 MB/s eta 0:00:01
   ------ --------------------------------- 1.6/9.5 MB 10.3 MB/s eta 0:00:01
   --------- ------------------------------ 2.4/9.5 MB 12.5 MB/s eta 0:00:01
   ------------- -------------------------- 3.3/9.5 MB 14.0 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.5 MB 15.7 MB/s eta 0:00:01
   -------------------- ------------------- 4.9/9.5 MB 15.8 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
import re

import time
import pandas as pd

In [2]:
chrome_path = "C:\\Users\\wmk51\\oliveyoung-project\\chromedriver-win64\\chromedriver.exe"
options = webdriver.ChromeOptions()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
options.add_argument("no-sandbox")
options.add_argument("disable-gpu")
options.add_argument("--lang=ko_KR")
options.add_argument('user-agent=' + user_agent)
driver = webdriver.Chrome(options=options)
url = "https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000160736&dispCatNo=90000010001&curation=goods_detail_a002&rccode=pc_detail_01_c&egcode=a002_a102&egrankcode=3&trackingCd=Curation1&t_page=%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_click=%EC%A7%80%EA%B8%88%EB%B9%84%EC%8A%B7%EC%83%81%ED%92%88_%EC%83%81%ED%92%88%EC%B6%94%EC%B2%9C_%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=2"
driver.get(url)
driver.set_window_size(1920,1080)
driver.maximize_window()


In [3]:
##최신 순으로 댓글 정렬    
html_url = driver.page_source
soup_url = BeautifulSoup(html_url,"html.parser")
time.sleep(2)
WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'goods_reputation'))
    ).click()
WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'filterBtn'))
    ).click()


In [4]:

WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="searchPoint"]/li[2]/label'))
    ).click()
WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="searchPoint"]/li[3]/label'))
    ).click()
WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnFilterConfirm'))
    ).click()


In [5]:

df_review_pos= pd.DataFrame(columns=['userId','rating','reviews'])

In [6]:

for current_page in range(1,31):
    print("현재 크롤링 페이지는 " ,current_page)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    time.sleep(2)
    products_info = soup.find("ul",attrs={"class" : "inner_list"})
    
    for product_info in products_info:
        
        userId = (product_info.find("p",attrs={"class":"info_user"}).get_text().split("  ")[0])
        rating = (int(product_info.find("span",attrs={"class":"point"}).get_text().split(" ")[1][0]))
        reviews = (product_info.find("div",attrs={"class":"txt_inner"}).get_text(),"\n")
        df_review_pos.loc[len(df_review_pos)] = [userId, rating, reviews]
    try:
        if current_page % 10 != 0: # 현재 페이지가 10의 배수가 아닐 때
            if current_page // 10 < 1: # 페이지 수가 한 자리수 일 때  
                # 리뷰 10개 긁으면 next 버튼 클릭
                page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({current_page%10+1})')
                page_button.click() 
                time.sleep(2)
            else: # 페이지 수가 두자리 수 이상일 때 
                # 리뷰 10개 긁으면 next 버튼 클릭
                page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({current_page%10+2})')
                page_button.click() 
                time.sleep(2)
        else:
            next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
            next_button.click() # 현재 페이지가 10의 배수일 때 페이지 넘김 버튼 클릭
            time.sleep(2)
    except:
            pass
    
        

    
    

# 3     
#  //*[@id="gdasContentsArea"]/div/div[8]/a[2]      
# 9페이
#  //*[@id="gdasContentsArea"]/div/div[8]/a[8]       

현재 크롤링 페이지는  1
현재 크롤링 페이지는  2
현재 크롤링 페이지는  3
현재 크롤링 페이지는  4
현재 크롤링 페이지는  5
현재 크롤링 페이지는  6
현재 크롤링 페이지는  7
현재 크롤링 페이지는  8
현재 크롤링 페이지는  9
현재 크롤링 페이지는  10
현재 크롤링 페이지는  11
현재 크롤링 페이지는  12
현재 크롤링 페이지는  13
현재 크롤링 페이지는  14
현재 크롤링 페이지는  15
현재 크롤링 페이지는  16
현재 크롤링 페이지는  17
현재 크롤링 페이지는  18
현재 크롤링 페이지는  19
현재 크롤링 페이지는  20
현재 크롤링 페이지는  21
현재 크롤링 페이지는  22
현재 크롤링 페이지는  23
현재 크롤링 페이지는  24
현재 크롤링 페이지는  25
현재 크롤링 페이지는  26
현재 크롤링 페이지는  27
현재 크롤링 페이지는  28
현재 크롤링 페이지는  29
현재 크롤링 페이지는  30


In [7]:
df_review_pos.head()

,userId,rating,reviews
0,와바밤,5,(사실 마스크팩을 한달동안 사용한것이 아니라 한달리뷰라고 하기 얘매하긴한데 어쩃든 ...
1,와바밤,5,"(배송비 가격 맞출때 유용하게 매번 구매하고 있습니다. 자극 없이 순해여, \n)"
2,보부상출신,5,"(나이 먹으니까 피부 고민이 점차 주름, 미백 쪽으로 빠지는데데일리로 피부결,톤 케..."
3,파이지니,5,(🌼1일1팩하기 좋은 가성비 마스크 팩🌼메디힐 마스크팩 너무 유명하죠:) 아무래도 ...
4,보부상출신,5,(요즘 제 피부상태 탄력이 지금 저세상 가있는데복구를 위해 콜라겐 마스크팩 이용해봅...


In [8]:
df_review_pos.to_csv("olive_young_pos_df")

In [232]:
df_review_pos.to_csv("ex_df")

In [231]:
df_review_pos.shape

(60, 3)

In [243]:
print(11%10)

1


In [6]:
print(10//10)

1
